# Gallery Example: H₂/M/1 Queue (Hyperexponential Arrivals)

This example demonstrates an H₂/M/1 queueing system:
- **Arrivals**: Hyperexponential distribution (more variable than Poisson)
- **Service**: Exponential service times
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

The Hyperexponential distribution has higher variance than exponential, representing more bursty/irregular arrivals with high variability (SCV=64).

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_hypm1(): Network {    """Create H₂/M/1 queueing model"""    model = Network("H2/M/1")        # Block 1: nodes    source = Source(model, "mySource")    queue = Queue(model, "myQueue", SchedStrategy.FCFS)    sink = Sink(model, "mySink")        # Block 2: classes    oclass = OpenClass(model, "myClass")    # Hyperexponential arrival with mean=1 and SCV=64 (highly variable)    source.setArrival(oclass, HyperExp.fit_mean_and_scv(1, 64))    # Exponential service with rate=2 (mean service time = 0.5)    queue.setService(oclass, Exp(2))        # Block 3: topology    P = model.initRoutingMatrix()    P.addRoute(oclass, source, queue, 1.0)    P.addRoute(oclass, queue, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_hypm1()

## Theoretical Analysis for H₂/M/1

For H₂/M/1 with:
- **Arrival rate**: λ = 1 (Hyperexponential with mean=1, SCV=64)
- **Service rate**: μ = 2 (exponential)
- **Utilization**: ρ = λ/μ = 0.5

The Hyperexponential distribution characteristics:
- **Mean**: 1.0
- **Squared Coefficient of Variation**: SCV = 64 (much higher than exponential SCV = 1)
- **Variance**: 64 (very high variability)

This high arrival variability typically results in much worse performance than M/M/1.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 20)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)

In [ ]:
// Compare H₂/M/1 with M/M/1 and Er/M/1
println("\n=== Comparison with M/M/1 and Er/M/1 ===")
// Create equivalent models
fun create_mm1_equivalent(): Network {
    val model_mm1 = Network("M/M/1-Equivalent")
    val source = Source(model_mm1, "Source")
    val queue = Queue(model_mm1, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_mm1, "Sink")
    
    val oclass = OpenClass(model_mm1, "Class")
    source.setArrival(oclass, Exp(1))  # SCV = 1
    queue.setService(oclass, Exp(2))
    
    val P = model_mm1.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_mm1.link(P)
    
    return model_mm1

fun create_erlm1_equivalent(): Network {
    val model_erl = Network("Er/M/1-Equivalent")
    val source = Source(model_erl, "Source")
    val queue = Queue(model_erl, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_erl, "Sink")
    
    val oclass = OpenClass(model_erl, "Class")
    source.setArrival(oclass, Erlang.fit_mean_and_order(1, 5))  # SCV = 0.2
    queue.setService(oclass, Exp(2))
    
    val P = model_erl.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_erl.link(P)
    
    return model_erl

val model_mm1 = create_mm1_equivalent()
val model_erl = create_erlm1_equivalent()

val solver_hyp = MVA(model)
val solver_mm1 = MVA(model_mm1)
val solver_erl = MVA(model_erl)

val avg_table_hyp = solver_hyp.avgTable
val avg_table_mm1 = solver_mm1.avgTable
val avg_table_erl = solver_erl.avgTable
// Extract key metrics
val hyp_resp = float(avg_table_hyp.iloc[1, 2])
val hyp_length = float(avg_table_hyp.iloc[1, 3])

val mm1_resp = float(avg_table_mm1.iloc[1, 2])
val mm1_length = float(avg_table_mm1.iloc[1, 3])

val erl_resp = float(avg_table_erl.iloc[1, 2])
val erl_length = float(avg_table_erl.iloc[1, 3])

println("Model Comparison (same λ=1, μ=2):")
println(f"Distribution | SCV  | Response Time | Queue Length")
println("-" * 50)
println(f"Erlang (k=5) | 0.20 |    {erl_resp:.4f}    |   {erl_length:.4f}")
println(f"Exponential  | 1.00 |    {mm1_resp:.4f}    |   {mm1_length:.4f}")
println(f"Hyperexp     |64.00 |    {hyp_resp:.4f}    |   {hyp_length:.4f}")

println(f"\nPerformance Impact:")
println(f"H₂/M/1 vs M/M/1: {(hyp_resp / mm1_resp):.1f}x worse response time")
println(f"Er/M/1 vs M/M/1: {(erl_resp / mm1_resp):.2f}x better response time")
println(f"Range: {(hyp_resp / erl_resp):.1f}x difference between high and low variability")

In [ ]:
// Analyze impact of arrival variability (SCV)
println("\n=== Impact of Arrival Variability ===")

fun create_hyperexp_model(scv): Network {
    """Create H₂/M/1 model with specified SCV"""
    if scv < 1:
// Use Erlang for SCV < 1
        model_var = Network(f"Er/M/1-SCV{scv}")
        val source = Source(model_var, "Source")
        val queue = Queue(model_var, "Queue", SchedStrategy.FCFS)
        val sink = Sink(model_var, "Sink")
        
        val oclass = OpenClass(model_var, "Class")
        val order = int(1.0 / scv)
        source.setArrival(oclass, Erlang.fit_mean_and_order(1, order))
        queue.setService(oclass, Exp(2))
    elif scv == 1:
// Use Exponential for SCV = 1
        model_var = Network(f"M/M/1-SCV{scv}")
        val source = Source(model_var, "Source")
        val queue = Queue(model_var, "Queue", SchedStrategy.FCFS)
        val sink = Sink(model_var, "Sink")
        
        val oclass = OpenClass(model_var, "Class")
        source.setArrival(oclass, Exp(1))
        queue.setService(oclass, Exp(2))
    else:
// Use Hyperexponential for SCV > 1
        model_var = Network(f"H2/M/1-SCV{scv}")
        val source = Source(model_var, "Source")
        val queue = Queue(model_var, "Queue", SchedStrategy.FCFS)
        val sink = Sink(model_var, "Sink")
        
        val oclass = OpenClass(model_var, "Class")
        source.setArrival(oclass, HyperExp.fit_mean_and_scv(1, scv))
        queue.setService(oclass, Exp(2))
    
    val P = model_var.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_var.link(P)
    
    return model_var
// Test different SCV values
val scv_values = [0.2, 0.5, 1.0, 4.0, 16.0, 64.0]

println("SCV  | Response Time | Queue Length | Performance Factor")
println("-" * 55)

val baseline_resp = None
for scv in scv_values:
    val model_scv = create_hyperexp_model(scv)
    val solver = MVA(model_scv)
    val avg_table = solver.avgTable
    
    val resp_time = float(avg_table.iloc[1, 2])
    val queue_length = float(avg_table.iloc[1, 3])
    
    if baseline_resp is None and scv == 1.0:
        val baseline_resp = resp_time
    
    val perf_factor = resp_time / baseline_resp if baseline_resp else 1.0
    
    println(f"{scv:4.1f} |    {resp_time:.4f}    |   {queue_length:.4f}    |      {perf_factor:.2f}")

println("\nNote: Higher SCV (more variability) leads to significantly worse performance.")
println("This demonstrates the fundamental impact of variability in queueing systems.")